#Read the datase

In [0]:
df = spark.table("workspace.bronze.epr_px")
df.display()

#Transformation

##remove unwanted spaces

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, trim, lit, regexp_replace, regexp_extract,StringType

In [0]:

for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))
        df.display()
   

##normalize maintenance flag to boolean


In [0]:

df = df.withColumn(
    "maintenance",
    F.when(F.upper(col("maintenance")) == "YES", F.lit(True))
     .when(F.upper(col("maintenance")) == "NO", F.lit(False))
     .otherwise(None)
)
df.display()


##rename column

In [0]:

col = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}
for old, new in col.items():
    df = df.withColumnRenamed(old, new)
    df.display()

#Writing silver table

In [0]:
df.write.mode("overwrite").saveAsTable("workspace.silver.epr_px")

##check table

In [0]:
%sql
select*from workspace.silver.epr_px